<a href="https://colab.research.google.com/github/Andicleomj/UAS-Machine-Learning-/blob/main/UAS/Member%203%3A%20NLP%20Hugging%20Face%20Course%20(Chapters%205-9)/Code%20Colab%20Member%203%3A%20NLP%20Hugging%20Face%20Course%20(Chapters%205-9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Member 3: NLP Hugging Face Course (Chapters 5-9)

In [61]:
!pip install datasets


In [62]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
import numpy as np

In [63]:
# Step 1: Load Dataset
dataset = load_dataset("glue", "mrpc")

In [64]:
# Step 2: Load Pretrained Model and Tokenizer
model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [65]:
# Step 3: Tokenize Dataset
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [66]:
# Step 4: Prepare for Training
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]

In [67]:
# Step 5: Define Metrics
metric = evaluate.load("glue", "mrpc")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [54]:
# Step 6: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

In [57]:
# Step 7: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-57-7313eba52d52>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [60]:
# Step 8: Train the Model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.370344,0.848039,0.891608
2,No log,0.434490,0.835784,0.884283
3,0.313200,0.531037,0.845588,0.891938


TrainOutput(global_step=690, training_loss=0.25695148689159447, metrics={'train_runtime': 10146.6441, 'train_samples_per_second': 1.084, 'train_steps_per_second': 0.068, 'total_flos': 529225276750320.0, 'train_loss': 0.25695148689159447, 'epoch': 3.0})

In [68]:
# Step 9: Evaluate the Model
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 0.37034448981285095, 'eval_accuracy': 0.8480392156862745, 'eval_f1': 0.8916083916083916, 'eval_runtime': 134.4885, 'eval_samples_per_second': 3.034, 'eval_steps_per_second': 0.193, 'epoch': 3.0}


In [69]:
# Step 10: Save the Model
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [70]:
# Step 11: Load the Saved Model for Inference
from transformers import pipeline
classifier = pipeline("text-classification", model="./saved_model", tokenizer="./saved_model")

Device set to use cpu


In [71]:
# Step 12: Run Inference
example = {"sentence1": "The company is doing great.", "sentence2": "The business is performing well."}
inputs = tokenizer(example["sentence1"], example["sentence2"], return_tensors="pt", truncation=True)
outputs = model(**inputs)
predictions = np.argmax(outputs.logits.detach().numpy(), axis=-1)
print("Inference result (label):", predictions)

Inference result (label): [1]
